In [ ]:
import sys

main_dir = "../"
if main_dir not in sys.path:
	sys.path.insert(0, main_dir)
 
import torch

from utils import cleanup, seed_everything
from network import GPTModel
from generation import continue_sentence, Strategies

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

/home/iahve/gvenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-09 09:20:29,373 - INFO - Device: cuda


Using device: cuda


# Masked inference

In [ ]:
masked = False
with_mask_state_dict = torch.load('runs/train_masked/gpt_baseline_with_mask.pth')
params = with_mask_state_dict['hyperparameters']

In [ ]:
vocab_size = params['vocab_size']
embed_dim = params['embed_dim']
num_layers = params['num_layers']
num_heads = params['num_heads']
ff_hidden_dim = params['ff_hidden_dim']

masked_model = GPTModel(vocab_size, embed_dim, num_layers, num_heads, ff_hidden_dim, use_causal_mask=masked).to(device)

In [9]:
masked_model.load_state_dict(with_mask_state_dict['model_state_dict'])

<All keys matched successfully>

In [ ]:
cleanup()
seed_everything(1337)
print('Greedy generation:')
greedy_sentence = continue_sentence("Я помню чудное мгновенье", masked_model, max_len=45, strategy=Strategies.greedy)
print(greedy_sentence, '\n')

print('Top k sampling generation:')

strategy_top_k = Strategies.top_k
strategy_top_k.param = 500
top_k_sentence = continue_sentence("Я помню чудное мгновенье", masked_model, max_len=45, strategy=strategy_top_k)
print(top_k_sentence, '\n')

print('Beam search generation:')
beam_sentence = continue_sentence("Я помню чудное мгновенье,", masked_model, max_len=45, strategy=Strategies.beam)
print(beam_sentence, '\n')

print('Random sample generation:')
random_sample_sentence = continue_sentence("Я помню чудное мгновенье,", masked_model, max_len=45, strategy=Strategies.random_sample)
print(random_sample_sentence)

Greedy generation:
я помню чудное мгновенье, и я не могу................................... 

Top 5 sampling generation:
я помню чудное мгновенье, сердце. а вот тебе теперь есть нечто приятное, змила? сорванья победоносно, и мне легче, я с тобою, ты мои, нам если б от души 

Beam search generation:
я помню чудное мгновенье, - сказал я.................................... 

Random sample generation:
я помню чудное мгновенье, как я умерла, и она теперь ; помнится и поступала, мирные, что все какие - то дальнеи губ. я должен уити.. а. я тебе,


In [19]:
strategy_top_k = Strategies.top_k
strategy_top_k.param = 100
print(continue_sentence("Мой дядя самых честных правил,", masked_model, max_len=45, strategy=strategy_top_k))
print(continue_sentence("Четыре года потратил Деонардо на", masked_model, max_len=45, strategy=strategy_top_k))
print(continue_sentence("Если сила плохих людей в том, что они вместе, то хорошим людям, чтобы стать силой, надо", masked_model, max_len=45, strategy=strategy_top_k))

мои дядя самых честных правил, - я должен был его любишь изъяснить. вот ты не то что на моем воспоминание, где жизнь, нет, и я сам был сам, как в уме моем кель
четыре года потратил деонардо на меня, на меня не пролишь ; а он вам отвечал. я не люблю, потому что будет угодно! я не могу, что еи ни один только что мог бы
если сила плохих людеи в том, что они вместе, то хорошим людям, чтобы стать силои, надо. ну, сам, я им теперь говорю ; я верю!. все эти товары


# Inference without mask

In [ ]:
masked = True
without_mask_state_dict = torch.load('runs/train_masked/gpt_baseline_without_mask.pth')
params = without_mask_state_dict['hyperparameters']

vocab_size = params['vocab_size']
embed_dim = params['embed_dim']
num_layers = params['num_layers']
num_heads = params['num_heads']
ff_hidden_dim = params['ff_hidden_dim']

maskless_model = GPTModel(vocab_size, embed_dim, num_layers, num_heads, ff_hidden_dim, use_causal_mask=masked).to(device)

maskless_model.load_state_dict(without_mask_state_dict['model_state_dict'])

<All keys matched successfully>

In [21]:
strategy_top_k = Strategies.top_k
strategy_top_k.param = 100
print(continue_sentence("Мой дядя самых честных правил,", maskless_model, max_len=45, strategy=strategy_top_k))
print(continue_sentence("Четыре года потратил Деонардо на", maskless_model, max_len=45, strategy=strategy_top_k))
print(continue_sentence("Если сила плохих людей в том, что они вместе, то хорошим людям, чтобы стать силой, надо", masked_model, max_len=45, strategy=strategy_top_k))

мои дядя самых честных правил, а не совсем даже никто другои и не был убежден. " умнистныи, когда - то никогда, а по нашему, что на первому не скажет, что человек
четыре года потратил деонардо на скашечныи чашу. - - - еи стансеи не выпуск в своеи, а в тоскную, которая может ехать, как бы в
если сила плохих людеи в том, что они вместе, то хорошим людям, чтобы стать силои, надо, помогать в них, а в то есть ; иначе быть может, и сами собес
